# raw_to_harmonized

In [36]:
# Import packages & setting up a connection to the ENTSOE API Client
import pandas as pd
import numpy as np

In [37]:
# Create a Local Reference
import os
current_dir = os.path.abspath("")
parent_dir = os.path.abspath(current_dir + "/../")
data_dir = os.path.abspath(parent_dir + "/data/raw/")
test_dir = os.path.abspath(parent_dir + "/data/test/")

print(current_dir)
print(parent_dir)
print(data_dir)

c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject\2. raw_to_harmonized
c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject
c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject\data\raw


In [74]:
# Read data file(s)
import os
import glob

for file in glob.glob(test_dir + "/*.csv"):
    df = pd.read_csv(
        file,
        delimiter = ",",
        # index_col = 0,
    )
    print(df.tail())

                      Unnamed: 0 Biomass Biomass.1 Fossil Brown coal/Lignite  \
34940  2017-12-30 22:45:00+01:00  4781.0       NaN                    6326.0   
34941  2017-12-30 23:00:00+01:00  4760.0       NaN                    6501.0   
34942  2017-12-30 23:15:00+01:00  4761.0       NaN                    6455.0   
34943  2017-12-30 23:30:00+01:00  4763.0       NaN                    6428.0   
34944  2017-12-30 23:45:00+01:00  4759.0       NaN                    6375.0   

      Fossil Brown coal/Lignite.1 Fossil Coal-derived gas  \
34940                         0.0                   520.0   
34941                         0.0                   520.0   
34942                         0.0                   491.0   
34943                         0.0                   491.0   
34944                         0.0                   491.0   

      Fossil Coal-derived gas.1 Fossil Gas Fossil Gas.1 Fossil Hard coal  ...  \
34940                       NaN     1154.0          1.0           1464.

C:\Users\SantiTaweesamarn\AppData\Local\Temp\ipykernel_18168\3086896339.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
